In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator

0it [00:00, ?it/s]
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [115]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [7]:
key = pd.read_csv('./key_trunc.csv')

In [8]:
sub = pd.read_csv("./sample_submission_1.csv")

In [257]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [136]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days, min_periods=7)[-60:]    

In [137]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days, min_periods=7)[-60:]    

In [138]:
def get_mean_diff(row, last_n_days):
    return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [139]:
def get_last_day_visits(row):
    return row[1:]

In [140]:
def get_long_stats(func_type, df, df_melt, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.progress_apply(get_median, last_n_days = last_n_days, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.progress_apply(get_mean, last_n_days = last_n_days, axis = 1)
        
#     elif func_type == "mean_diff":
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_mean_diff, last_n_days = last_n_days, axis = 1)
        
#     elif func_type == "last_day_visits":        
#         rolling_stats = df.iloc[:, -61:].progress_apply(get_last_day_visits, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [141]:
def generate_train_melt(train):
    
#     train_melt = pd.read_csv("train_feats.csv")
    
#     Commented because these features are already generated
    train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

#     train_melt = get_long_stats("mean", train, train_melt, 7)
    train_melt = get_long_stats("mean", train, train_melt, 30)
    train_melt = get_long_stats("mean", train, train_melt, 60)

#     train_melt = get_long_stats("median", train, train_melt, 7)
    train_melt = get_long_stats("median", train, train_melt, 30)
    train_melt = get_long_stats("median", train, train_melt, 60)

#     train_melt = get_long_stats("mean_diff", train, train_melt, 7)
#     train_melt = get_long_stats("last_day_visits", train, train_melt)
    
    train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
    
    # Columns i am not using
    # train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
    # train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)
    
    return train_melt 

train_melt = generate_train_melt(train)
train_melt.to_csv("train_feats.csv", index = False)







  0%|          | 0/145063 [00:00<?, ?it/s]





  0%|          | 1/145063 [00:10<422:04:59, 10.47s/it]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(min_periods=7,window=30,center=False).mean()
  






  0%|          | 2/145063 [00:15<357:59:54,  8.88s/it]





  0%|          | 125/145063 [00:15<250:23:46,  6.22s/it]





  0%|          | 260/145063 [00:15<175:07:23,  4.35s/it]





  0%|          | 395/145063 [00:15<122:28:50,  3.05s/it]





  0%|          | 528/145063 [00:16<85:40:00,  2.13s/it] 





  0%|          | 671/145063 [00:16<59:54:57,  1.49s/it]





  1%|          | 805/145063 [00:16<41:54:40,  1.05s/it]





  1%|          | 925/145063 [00:16<29:19:24,  1.37it/s]





  1%|          | 1058/145063 [00:16<20:30:59,  1.95it/s]





  1%|          | 1193/145063 [00:16<14:21:25,  2.78it/s]





  1%|          | 1324/145063 [00:16<1

 10%|▉         | 13968/145063 [00:25<01:33, 1401.03it/s]





 10%|▉         | 14116/145063 [00:25<01:32, 1420.92it/s]





 10%|▉         | 14264/145063 [00:25<01:30, 1437.71it/s]





 10%|▉         | 14413/145063 [00:25<01:30, 1449.89it/s]





 10%|█         | 14575/145063 [00:26<01:27, 1495.60it/s]





 10%|█         | 14726/145063 [00:26<01:29, 1459.79it/s]





 10%|█         | 14873/145063 [00:26<01:29, 1461.61it/s]





 10%|█         | 15020/145063 [00:26<01:30, 1430.01it/s]





 10%|█         | 15176/145063 [00:26<01:28, 1465.24it/s]





 11%|█         | 15335/145063 [00:26<01:26, 1500.19it/s]





 11%|█         | 15486/145063 [00:26<01:33, 1383.13it/s]





 11%|█         | 15627/145063 [00:26<01:36, 1338.77it/s]





 11%|█         | 15763/145063 [00:26<01:38, 1308.93it/s]





 11%|█         | 15911/145063 [00:27<01:35, 1353.83it/s]





 11%|█         | 16060/145063 [00:27<01:32, 1391.58it/s]





 11%|█         | 16223/145063 [00:27<01:28, 1455.05it/s]





 11%|█▏ 

 21%|██        | 29980/145063 [00:36<01:17, 1481.99it/s]





 21%|██        | 30130/145063 [00:36<01:17, 1485.19it/s]





 21%|██        | 30279/145063 [00:36<01:17, 1474.49it/s]





 21%|██        | 30434/145063 [00:36<01:16, 1494.27it/s]





 21%|██        | 30584/145063 [00:36<01:16, 1489.97it/s]





 21%|██        | 30734/145063 [00:36<01:17, 1476.23it/s]





 21%|██▏       | 30883/145063 [00:36<01:17, 1478.23it/s]





 21%|██▏       | 31031/145063 [00:36<01:18, 1454.21it/s]





 21%|██▏       | 31180/145063 [00:37<01:17, 1462.60it/s]





 22%|██▏       | 31327/145063 [00:39<09:12, 205.70it/s] 





 22%|██▏       | 31501/145063 [00:39<06:46, 279.65it/s]





 22%|██▏       | 31670/145063 [00:39<05:03, 373.03it/s]





 22%|██▏       | 31837/145063 [00:39<03:52, 486.27it/s]





 22%|██▏       | 32003/145063 [00:39<03:03, 616.93it/s]





 22%|██▏       | 32169/145063 [00:39<02:28, 759.66it/s]





 22%|██▏       | 32340/145063 [00:39<02:03, 910.91it/s]





 22%|██▏      

 32%|███▏      | 46161/145063 [00:49<01:07, 1471.60it/s]





 32%|███▏      | 46317/145063 [00:49<01:06, 1494.19it/s]





 32%|███▏      | 46468/145063 [00:49<01:05, 1497.31it/s]





 32%|███▏      | 46618/145063 [00:49<01:05, 1494.39it/s]





 32%|███▏      | 46769/145063 [00:50<01:05, 1497.76it/s]





 32%|███▏      | 46919/145063 [00:50<01:05, 1496.17it/s]





 32%|███▏      | 47069/145063 [00:50<01:05, 1486.00it/s]





 33%|███▎      | 47218/145063 [00:50<01:06, 1475.54it/s]





 33%|███▎      | 47366/145063 [00:50<01:06, 1467.98it/s]





 33%|███▎      | 47513/145063 [00:50<01:06, 1458.17it/s]





 33%|███▎      | 47665/145063 [00:50<01:06, 1474.91it/s]





 33%|███▎      | 47817/145063 [00:50<01:05, 1486.62it/s]





 33%|███▎      | 47970/145063 [00:50<01:04, 1497.03it/s]





 33%|███▎      | 48120/145063 [00:50<01:06, 1466.95it/s]





 33%|███▎      | 48275/145063 [00:51<01:04, 1490.11it/s]





 33%|███▎      | 48431/145063 [00:51<01:04, 1509.32it/s]





 33%|███

 42%|████▏     | 61547/145063 [01:01<00:54, 1518.98it/s]





 43%|████▎     | 61699/145063 [01:01<00:56, 1488.00it/s]





 43%|████▎     | 61848/145063 [01:01<00:56, 1463.87it/s]





 43%|████▎     | 61995/145063 [01:01<00:56, 1458.63it/s]





 43%|████▎     | 62144/145063 [01:01<00:56, 1466.51it/s]





 43%|████▎     | 62291/145063 [01:01<00:57, 1446.03it/s]





 43%|████▎     | 62442/145063 [01:01<00:56, 1463.78it/s]





 43%|████▎     | 62589/145063 [01:01<00:56, 1454.06it/s]





 43%|████▎     | 62736/145063 [01:01<00:56, 1458.72it/s]





 43%|████▎     | 62887/145063 [01:02<00:55, 1470.71it/s]





 43%|████▎     | 63045/145063 [01:02<00:54, 1501.14it/s]





 44%|████▎     | 63201/145063 [01:02<00:53, 1516.96it/s]





 44%|████▎     | 63353/145063 [01:02<00:54, 1486.68it/s]





 44%|████▍     | 63513/145063 [01:02<00:53, 1518.16it/s]





 44%|████▍     | 63672/145063 [01:02<00:52, 1535.69it/s]





 44%|████▍     | 63826/145063 [01:02<00:53, 1531.50it/s]





 44%|███

...

 87%|████████▋ | 7533231/8703780 [05:14<00:49, 23826.68it/s]





 87%|████████▋ | 7535681/8703780 [05:14<00:48, 24024.06it/s]





 87%|████████▋ | 7538088/8703780 [05:14<00:48, 24032.49it/s]





 87%|████████▋ | 7540495/8703780 [05:14<00:49, 23597.92it/s]





 87%|████████▋ | 7542933/8703780 [05:14<00:48, 23826.95it/s]





 87%|████████▋ | 7545319/8703780 [05:15<00:48, 23704.22it/s]





 87%|████████▋ | 7547716/8703780 [05:15<00:48, 23781.50it/s]





 87%|████████▋ | 7550119/8703780 [05:15<00:48, 23854.16it/s]





 87%|████████▋ | 7552518/8703780 [05:15<00:48, 23891.83it/s]





 87%|████████▋ | 7554909/8703780 [05:15<00:49, 23119.48it/s]





 87%|████████▋ | 7557227/8703780 [05:15<00:49, 23089.97it/s]





 87%|████████▋ | 7559659/8703780 [05:15<00:48, 23444.02it/s]





 87%|████████▋ | 7562052/8703780 [05:15<00:48, 23587.65it/s]





 87%|████████▋ | 7564414/8703780 [05:15<00:48, 23582.22it/s]





 87%|████████▋ | 7567087/8703780 [05:15<00:46, 24444.04it/s]





 87%|█████

 89%|████████▉ | 7766748/8703780 [05:24<00:38, 24058.56it/s]





 89%|████████▉ | 7769236/8703780 [05:24<00:38, 24296.24it/s]





 89%|████████▉ | 7771669/8703780 [05:24<00:38, 23961.87it/s]





 89%|████████▉ | 7774069/8703780 [05:24<00:38, 23927.18it/s]





 89%|████████▉ | 7776464/8703780 [05:24<00:39, 23315.26it/s]





 89%|████████▉ | 7778801/8703780 [05:24<00:39, 23143.74it/s]





 89%|████████▉ | 7781120/8703780 [05:25<00:40, 22999.33it/s]





 89%|████████▉ | 7783423/8703780 [05:25<00:40, 22497.72it/s]





 89%|████████▉ | 7785730/8703780 [05:25<00:40, 22664.34it/s]





 89%|████████▉ | 7788265/8703780 [05:25<00:39, 23406.38it/s]





 90%|████████▉ | 7790679/8703780 [05:25<00:38, 23621.67it/s]





 90%|████████▉ | 7793048/8703780 [05:25<00:38, 23445.44it/s]





 90%|████████▉ | 7795512/8703780 [05:25<00:38, 23789.45it/s]





 90%|████████▉ | 7797896/8703780 [05:25<00:38, 23630.53it/s]





 90%|████████▉ | 7800314/8703780 [05:25<00:37, 23791.75it/s]





 90%|█████

 92%|█████████▏| 8001684/8703780 [05:34<00:30, 22965.06it/s]





 92%|█████████▏| 8004183/8703780 [05:34<00:29, 23535.11it/s]





 92%|█████████▏| 8006625/8703780 [05:34<00:29, 23792.02it/s]





 92%|█████████▏| 8009011/8703780 [05:34<00:29, 23486.10it/s]





 92%|█████████▏| 8011365/8703780 [05:34<00:29, 23464.01it/s]





 92%|█████████▏| 8013716/8703780 [05:34<00:30, 22848.25it/s]





 92%|█████████▏| 8016012/8703780 [05:35<00:30, 22879.34it/s]





 92%|█████████▏| 8018424/8703780 [05:35<00:29, 23237.39it/s]





 92%|█████████▏| 8020753/8703780 [05:35<00:29, 22800.50it/s]





 92%|█████████▏| 8023275/8703780 [05:35<00:28, 23475.78it/s]





 92%|█████████▏| 8025654/8703780 [05:35<00:28, 23567.49it/s]





 92%|█████████▏| 8028165/8703780 [05:35<00:28, 24009.14it/s]





 92%|█████████▏| 8030573/8703780 [05:35<00:28, 23920.78it/s]





 92%|█████████▏| 8032970/8703780 [05:35<00:28, 23292.32it/s]





 92%|█████████▏| 8035324/8703780 [05:35<00:28, 23365.80it/s]





 92%|█████

 95%|█████████▍| 8237393/8703780 [05:44<00:19, 23822.18it/s]





 95%|█████████▍| 8239875/8703780 [05:44<00:19, 24111.95it/s]





 95%|█████████▍| 8242401/8703780 [05:44<00:18, 24444.25it/s]





 95%|█████████▍| 8244873/8703780 [05:44<00:18, 24524.30it/s]





 95%|█████████▍| 8247328/8703780 [05:44<00:18, 24404.26it/s]





 95%|█████████▍| 8249786/8703780 [05:44<00:18, 24456.13it/s]





 95%|█████████▍| 8252233/8703780 [05:44<00:18, 23872.75it/s]





 95%|█████████▍| 8254787/8703780 [05:45<00:18, 24345.37it/s]





 95%|█████████▍| 8257227/8703780 [05:45<00:18, 23972.98it/s]





 95%|█████████▍| 8259726/8703780 [05:45<00:18, 24266.53it/s]





 95%|█████████▍| 8262157/8703780 [05:45<00:18, 24116.47it/s]





 95%|█████████▍| 8264572/8703780 [05:45<00:18, 24101.65it/s]





 95%|█████████▍| 8267169/8703780 [05:45<00:17, 24632.46it/s]





 95%|█████████▌| 8269679/8703780 [05:45<00:17, 24769.28it/s]





 95%|█████████▌| 8272160/8703780 [05:45<00:17, 24395.58it/s]





 95%|█████

 97%|█████████▋| 8474509/8703780 [05:54<00:09, 23837.15it/s]





 97%|█████████▋| 8476895/8703780 [05:54<00:09, 23592.05it/s]





 97%|█████████▋| 8479363/8703780 [05:54<00:09, 23906.07it/s]





 97%|█████████▋| 8481757/8703780 [05:54<00:09, 23345.99it/s]





 97%|█████████▋| 8484097/8703780 [05:54<00:09, 23046.57it/s]





 98%|█████████▊| 8486536/8703780 [05:54<00:09, 23430.55it/s]





 98%|█████████▊| 8488958/8703780 [05:54<00:09, 23660.46it/s]





 98%|█████████▊| 8491328/8703780 [05:54<00:09, 23589.40it/s]





 98%|█████████▊| 8493690/8703780 [05:55<00:09, 23211.76it/s]





 98%|█████████▊| 8496065/8703780 [05:55<00:08, 23368.94it/s]





 98%|█████████▊| 8498405/8703780 [05:55<00:08, 23071.70it/s]





 98%|█████████▊| 8500725/8703780 [05:55<00:08, 23109.29it/s]





 98%|█████████▊| 8503084/8703780 [05:55<00:08, 23251.16it/s]





 98%|█████████▊| 8505519/8703780 [05:55<00:08, 23566.98it/s]





 98%|█████████▊| 8508017/8703780 [05:55<00:08, 23973.52it/s]





 98%|█████

In [7]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [8]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [142]:
def get_month(row):
    return row.date.split("-")[1]

In [143]:
def get_date(row):
    return row.date.split("-")[2]

In [144]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [193]:
def get_median_test(row, last_index, last_n_days, pred_index):
    
    start_loc = max(min(last_index - last_n_days, pred_index), 0)
    end_loc = min(last_index, pred_index)

    if start_loc < end_loc:
        return np.median(row.iloc[ start_loc : end_loc ].tolist())
    else:
        return np.median(row.iloc[ pred_index - last_n_days : pred_index ].tolist())


In [194]:
def get_mean_test(row, last_index, last_n_days, pred_index):
    
    start_loc = max(min(last_index - last_n_days, pred_index), 0)
    end_loc = min(last_index, pred_index)
    
    if start_loc < end_loc:
        return np.mean(row.iloc[start_loc : end_loc].tolist())
    else:
        return np.mean(row.iloc[pred_index - last_n_days : pred_index].tolist())

In [147]:
def get_mean_diff_test(row, last_index):
    
    start_loc = min(last_index - 7, 61)
    end_loc = min(last_index, 61)
    
    if start_loc < end_loc:
        return np.mean([int(x) for x in np.diff( row.iloc[start_loc : end_loc] ) ], dtype=None)
    else:
        return np.nan

In [148]:
def get_last_day_visits_test(row, last_index):
    
    return row.iloc[last_index - 1]

In [181]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [240]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0
    
    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 20, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)

    return bst

In [258]:
# label_train_melt = np.log1p(train_melt["visits"])
label_train_melt = train_melt["visits"]

# label_train_melt = [0] + np.diff(train_melt["visits"]).tolist()

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

# train_melt.drop("last_day_visits_None", axis = 1, inplace = True)
# train_melt.drop("mean_diff_7", axis = 1, inplace = True)

# train_melt.drop("mean_7", axis = 1, inplace = True)
# train_melt.drop("median_7", axis = 1, inplace = True)

train_melt.date_num = pd.to_numeric(train_melt.date_num)

In [241]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
bst = run_xgb(x_train, label_train, x_valid, label_valid)

# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:4.57713	validation-rmse:4.57638	train-smape:137.699	validation-smape:142.897
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:2.74194	validation-rmse:2.74154	train-smape:68.7948	validation-smape:70.9061
[19]	train-rmse:1.73024	validation-rmse:1.73003	train-smape:41.5817	validation-smape:43.4366


In [242]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

    feature    fscore
0   weekday  0.004766
1  date_num  0.010326
2   mean_30  0.984909


In [232]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

# x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
# # x_train = x_train.fillna(0)
# # x_valid = x_valid.fillna(0)
# reg.fit(x_train, label_train)

reg.fit(train_melt, label_train_melt)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [233]:
# prediction = reg.predict(x_valid)
prediction = reg.predict(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]])

In [223]:
y_true = np.array(label_valid)
y_pred = np.array(prediction)

denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
diff = np.abs(y_true - y_pred) / denominator
diff[denominator == 0] = 0.0

print np.nanmean(diff)

47.1048674182


In [189]:
def get_preds(test_df):
    
    d_test = xgb.DMatrix(test_df)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [190]:
def get_overall_df():
    
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [191]:
def get_test_df(overall_df):
    
    test_df = pd.DataFrame()

    pred_index = overall_df.columns.tolist().index('2017-01-01')
    
    for i in range(pred_index, len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date
        last_index = i

        test_feats = pd.DataFrame()
        
        test_feats["Page"] = overall_df.Page
        test_feats["date"] = last_date
        
#         test_feats["mean_7"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 7, pred_index = pred_index)
        test_feats["mean_30"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 30, pred_index = pred_index)
        test_feats["mean_60"] = overall_df.apply(get_mean_test, axis = 1, last_index = last_index, last_n_days = 60, pred_index = pred_index)

#         test_feats["median_7"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 7, pred_index = pred_index)
        test_feats["median_30"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 30, pred_index = pred_index)
        test_feats["median_60"] = overall_df.apply(get_median_test, axis = 1, last_index = last_index, last_n_days = 60, pred_index = pred_index)

#         test_feats["mean_diff_7"] = overall_df.apply(get_mean_diff_test, axis = 1, last_index = last_index)
#         test_feats["last_day_visits_None"] = overall_df.apply(get_last_day_visits_test, axis = 1, last_index = last_index)

#         test_feats["month"] = last_date.split("-")[1]
        test_feats["date_num"] = pd.to_numeric(last_date.split("-")[2])
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()
                                
        test_df = pd.concat([test_df, test_feats])
        
    return test_df

In [195]:
overall_df = get_overall_df()
test_df = get_test_df(overall_df)
test_df.to_csv("test_feats.csv", index = False)

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-14
2017-01-15
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-21
2017-01-22
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-01-27
2017-01-28
2017-01-29
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-04
2017-02-05
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-11
2017-02-12
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-18
2017-02-19
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-25
2017-02-26
2017-02-27
2017-02-28
2017-03-01


In [197]:
# test_df = pd.read_csv("test_feats.csv")
# prediction = get_preds(test_df[["mean_7","mean_30","mean_60","median_7","median_30","median_60","date_num", "weekday"]])
prediction = get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]])

In [226]:
# pred_index = overall_df.columns.tolist().index("2017-01-01")

# pred_df = overall_df.iloc[:, pred_index:]
# pred_df["Page"] = overall_df.Page

# pred_df_melt = pd.melt(pred_df, id_vars=['Page'], var_name="date", value_name="Visits")        

test_df["Visits"] = prediction
# pred_df = pd.concat([test_df[["Page", "date"]], prediction], axis = 1)
pred_df = test_df[["Page", "date", "Visits"]]

pred_id_visits = pd.merge(pred_df, key, left_on=["Page", "date"], right_on=["trunc_page", "date"], how="inner")[["Id", "Visits"]]

pred_id_visits.to_csv("pred_glm_2.csv", index = False)



In [288]:
from scipy.stats import *
bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt if i > 4 and i < 1871])) #Outlier 10% to 90% only
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt]))

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/morestats.py:516: RuntimeWarning: divide by zero encountered in power
  y = where(lmbda == 0, log(x), (x**lmbda - 1)/lmbda)


In [291]:
# np.percentile(label_train_melt, 90)
# np.percentile(label_train_melt, 10)
# len([i for i in label_train_melt if i > 4 and i < 1871])

import matplotlib.pyplot as plt
plt.hist(bc_label.tolist())
plt.show()

8.4721358117221772